In [1]:
from utils import get_path, group_cols
from datatable import dt, f, by, as_type

# Descrição

Neste notebook iremos formar uma tabela contendo todos os municípios de residência que estão na base do SIH.

# Tabelas base

## Localidades

### Colunas da tabela

In [2]:
path_loc = get_path('HEALTH', 'localidades.csv.gzip')
df_loc = dt.fread(path_loc)
df_loc = df_loc[:, f[:].remove(f.nome_regiao_saude)]
df_loc

,cod_municipio,nome_municipio,capital,cod_uf,uf,nome_uf,regiao,latitude,longitude
,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,520005,Abadia de Goiás,0,52,GO,Goiás,Centro-Oeste,−16.7573,−49.4412
1,310010,Abadia dos Dourados,0,31,MG,Minas Gerais,Sudeste,−18.4831,−47.3916
2,520010,Abadiânia,0,52,GO,Goiás,Centro-Oeste,−16.197,−48.7057
3,310020,Abaeté,0,31,MG,Minas Gerais,Sudeste,−19.1551,−45.4444
4,150010,Abaetetuba,0,15,PA,Pará,Norte,−1.72183,−48.8788
5,230010,Abaiara,0,23,CE,Ceará,Nordeste,−7.34588,−39.0416
6,290010,Abaíra,0,29,BA,Bahia,Nordeste,−13.2488,−41.6619
7,290020,Abaré,0,29,BA,Bahia,Nordeste,−8.72073,−39.1162
8,410010,Abatiá,0,41,PR,Paraná,Sul,−23.3049,−50.3133


## Indicadores

### Colunas da tabela

In [3]:
cols_indc = {
  'CÓDIGO': 'codigo',
  'Tipo Localidade': 'tipo',
  'CIR': 'nome_regiao_saude',
  'Grupos de Regiões de Saúde segundo Condições Socioeconômicas e de Saúde - 2016': 'socioecon',
  ...: None
}

In [4]:
path_indc = path_indc = get_path('INDICADORES', 'indicadores.csv.gzip')
df_indc = dt.fread(path_indc, columns=cols_indc)
df_indc = df_indc.to_pandas()
df_indc

,codigo,tipo,nome_regiao_saude,socioecon
0,15.001,REGSAU,Araguaia,1
1,15.002,REGSAU,Baixo Amazonas,1
2,15.003,REGSAU,Carajás,2
3,15.004,REGSAU,Lago de Tucuruí,1
4,15.013,REGSAU,Marajó I,1
...,...,...,...,...
6003,171.780,MUNIC,Sudeste,1
6004,171.800,MUNIC,Sudeste,1
6005,171.865,MUNIC,Sudeste,1
6006,172.090,MUNIC,Sudeste,1


### Municípios

In [5]:
df_munic = df_indc.loc[df_indc['tipo'] == 'MUNIC'].copy()
df_munic.drop(columns=['tipo'], inplace=True)
df_munic['codigo'] = df_munic['codigo'].apply(lambda x: (str(x).replace('.', '') + '00')[:6])
df_munic['prefix'] = df_munic['codigo'].apply(lambda x: str(x)[:2])
df_munic = df_munic.rename(columns={'codigo': 'cod_municipio'})
df_munic


,cod_municipio,nome_regiao_saude,socioecon,prefix
13,310600,João Monlevade,3,31
14,313620,João Monlevade,3,31
15,314470,João Monlevade,3,31
16,315570,João Monlevade,3,31
17,316100,João Monlevade,3,31
...,...,...,...,...
6003,171780,Sudeste,1,17
6004,171800,Sudeste,1,17
6005,171865,Sudeste,1,17
6006,172090,Sudeste,1,17


### Regiões de Saúde

In [6]:
df_regsau = df_indc[df_indc['tipo'] == 'REGSAU'].copy()
df_regsau.drop(columns=['tipo', 'socioecon'], inplace=True)
df_regsau['codigo'] = df_regsau['codigo'].apply(lambda x: (str(x).replace('.', '') + '0')[:5])
df_regsau['prefix'] = df_regsau['codigo'].apply(lambda x: str(x)[:2])
df_regsau = df_regsau.rename(columns={'codigo': 'cod_regiao_saude'})
df_regsau

,cod_regiao_saude,nome_regiao_saude,prefix
0,15001,Araguaia,15
1,15002,Baixo Amazonas,15
2,15003,Carajás,15
3,15004,Lago de Tucuruí,15
4,15013,Marajó I,15
...,...,...,...
5431,17001,Médio Norte Araguaia,17
5432,28005,Nossa Senhora da Glória,28
5433,28006,Nossa Senhora do Socorro,28
5434,28007,Propriá,28


### Merge

In [7]:
df_indc = df_munic.merge(df_regsau, how='left', left_on=['prefix', 'nome_regiao_saude'], right_on=['prefix', 'nome_regiao_saude'])
df_indc = dt.Frame(df_indc)
df_indc[:, 'cod_municipio'] = df_indc[:, dt.as_type(f.cod_municipio, int)]
df_indc

,cod_municipio,nome_regiao_saude,socioecon,prefix,cod_regiao_saude
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,310600,João Monlevade,3,31,31035
1,313620,João Monlevade,3,31,31035
2,314470,João Monlevade,3,31,31035
3,315570,João Monlevade,3,31,31035
4,316100,João Monlevade,3,31,31035
5,310855,João Pinheiro,2,31,31036
6,313630,João Pinheiro,2,31,31036
7,313753,João Pinheiro,2,31,31036
8,310280,Juiz de Fora Lima Duarte Bom Jardim Min,5,31,31037


# Gerando tabela

## Juntando com outras

### Juntando com Indicadores

In [8]:
df_indc.key = 'cod_municipio'
df_muns = df_loc[:, :, dt.join(df_indc)]
df_muns

,cod_municipio,nome_municipio,capital,cod_uf,uf,nome_uf,regiao,latitude,longitude,nome_regiao_saude,socioecon,prefix,cod_regiao_saude
,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,520005,Abadia de Goiás,0,52,GO,Goiás,Centro-Oeste,−16.7573,−49.4412,Central,5,52,52001
1,310010,Abadia dos Dourados,0,31,MG,Minas Gerais,Sudeste,−18.4831,−47.3916,Patrocínio Monte Carmelo,2,31,31052
2,520010,Abadiânia,0,52,GO,Goiás,Centro-Oeste,−16.197,−48.7057,Pirineus,3,52,52011
3,310020,Abaeté,0,31,MG,Minas Gerais,Sudeste,−19.1551,−45.4444,Sete Lagoas,3,31,31067
4,150010,Abaetetuba,0,15,PA,Pará,Norte,−1.72183,−48.8788,Tocantins,1,15,15011
5,230010,Abaiara,0,23,CE,Ceará,Nordeste,−7.34588,−39.0416,19ª Região Brejo Santo,1,23,23019
6,290010,Abaíra,0,29,BA,Bahia,Nordeste,−13.2488,−41.6619,Seabra,1,29,29023
7,290020,Abaré,0,29,BA,Bahia,Nordeste,−8.72073,−39.1162,Paulo Afonso,1,29,29017
8,410010,Abatiá,0,41,PR,Paraná,Sul,−23.3049,−50.3133,18ª RS Cornélio Procópio,3,41,41018


## Ajustando colunas

In [9]:
cols_order = [
  'cod_municipio',
  'nome_municipio',
  'capital',
  'cod_regiao_saude',
  'nome_regiao_saude',
  'cod_uf',
  'uf',
  'nome_uf',
  'regiao',
  'socioecon',
  'latitude',
  'longitude',
]

In [10]:
df_muns = df_muns[:, cols_order]
df_muns

,cod_municipio,nome_municipio,capital,cod_regiao_saude,nome_regiao_saude,cod_uf,uf,nome_uf,regiao,socioecon,latitude,longitude
,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,520005,Abadia de Goiás,0,52001,Central,52,GO,Goiás,Centro-Oeste,5,−16.7573,−49.4412
1,310010,Abadia dos Dourados,0,31052,Patrocínio Monte Carmelo,31,MG,Minas Gerais,Sudeste,2,−18.4831,−47.3916
2,520010,Abadiânia,0,52011,Pirineus,52,GO,Goiás,Centro-Oeste,3,−16.197,−48.7057
3,310020,Abaeté,0,31067,Sete Lagoas,31,MG,Minas Gerais,Sudeste,3,−19.1551,−45.4444
4,150010,Abaetetuba,0,15011,Tocantins,15,PA,Pará,Norte,1,−1.72183,−48.8788
5,230010,Abaiara,0,23019,19ª Região Brejo Santo,23,CE,Ceará,Nordeste,1,−7.34588,−39.0416
6,290010,Abaíra,0,29023,Seabra,29,BA,Bahia,Nordeste,1,−13.2488,−41.6619
7,290020,Abaré,0,29017,Paulo Afonso,29,BA,Bahia,Nordeste,1,−8.72073,−39.1162
8,410010,Abatiá,0,41018,18ª RS Cornélio Procópio,41,PR,Paraná,Sul,3,−23.3049,−50.3133


# Exportando tabela

In [11]:
output_muns = get_path('GESTANTES', 'municipios.csv.gzip')
df_muns.to_csv(output_muns)

In [12]:
dt.fread(output_muns)

,cod_municipio,nome_municipio,capital,cod_regiao_saude,nome_regiao_saude,cod_uf,uf,nome_uf,regiao,socioecon,latitude,longitude
,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,520005,Abadia de Goiás,0,52001,Central,52,GO,Goiás,Centro-Oeste,5,−16.7573,−49.4412
1,310010,Abadia dos Dourados,0,31052,Patrocínio Monte Carmelo,31,MG,Minas Gerais,Sudeste,2,−18.4831,−47.3916
2,520010,Abadiânia,0,52011,Pirineus,52,GO,Goiás,Centro-Oeste,3,−16.197,−48.7057
3,310020,Abaeté,0,31067,Sete Lagoas,31,MG,Minas Gerais,Sudeste,3,−19.1551,−45.4444
4,150010,Abaetetuba,0,15011,Tocantins,15,PA,Pará,Norte,1,−1.72183,−48.8788
5,230010,Abaiara,0,23019,19ª Região Brejo Santo,23,CE,Ceará,Nordeste,1,−7.34588,−39.0416
6,290010,Abaíra,0,29023,Seabra,29,BA,Bahia,Nordeste,1,−13.2488,−41.6619
7,290020,Abaré,0,29017,Paulo Afonso,29,BA,Bahia,Nordeste,1,−8.72073,−39.1162
8,410010,Abatiá,0,41018,18ª RS Cornélio Procópio,41,PR,Paraná,Sul,3,−23.3049,−50.3133


In [13]:
df_muns.names

('cod_municipio',
 'nome_municipio',
 'capital',
 'cod_regiao_saude',
 'nome_regiao_saude',
 'cod_uf',
 'uf',
 'nome_uf',
 'regiao',
 'socioecon',
 'latitude',
 'longitude')

http://cnes.datasus.gov.br/pages/estabelecimentos/consulta.jsp?search=2006405

In [14]:
scales = ['Região', 'UF', 'Regional Saúde', 'Município', 'Hospital']